In [1]:
import random   
import torch
import logging
import numpy as np
import gc


def set_seed(seed):
    logger = logging.getLogger("Utils")
    logger.debug(f"Setting seed to {seed}")
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def check_and_clear_memory():
    logger = logging.getLogger("Utils")
    logger.debug(f'Memória alocada antes da limpeza: {torch.cuda.memory_allocated()} bytes')
    logger.debug(f'Memória reservada antes da limpeza: {torch.cuda.memory_reserved()} bytes')

    torch.cuda.empty_cache()
    gc.collect()

    logger.debug(f'Memória alocada após limpeza: {torch.cuda.memory_allocated()} bytes')
    logger.debug(f'Memória reservada após limpeza: {torch.cuda.memory_reserved()} bytes')
    
    

In [2]:
import kornia
from kornia.feature import LocalFeature, LAFDescriptor, MultiResolutionDetector,SOSNet
from kornia.feature import CornerGFTT, PassLAF, LAFOrienter, LAFAffNetShapeEstimator
from kornia.feature.scale_space_detector import get_default_detector_config
from kornia.feature.keynet import KeyNetDetector
from kornia.feature.keynet import keynet_default_config

from external.REKD import REKD
from teste_util import CustomNetDetector

class GFTTFeatureSosNet(LocalFeature): #0.9    |   0.005
    """Convenience module, which implements GFTT detector + SOSNet descriptor."""

    def __init__(
        self,
        num_features: int = 200,
        upright: bool = False,
        device: torch.device = torch.device("cpu"),
        config: dict = None,
    ) -> None:
        if config is None:
            config = get_default_detector_config()
        detector = MultiResolutionDetector(
            CornerGFTT(),
            num_features,
            config,
            ori_module=PassLAF() if upright else LAFOrienter(19),
            aff_module=LAFAffNetShapeEstimator(preserve_orientation=upright).eval(),  # Usa `upright` para definir `preserve_orientation`
        ).to(device)

        # Initialize your descriptor (e.g., SOSNet) as before
        # Example with SOSNet - replace with actual initialization if different
        sosnet32 = SOSNet(pretrained=True)  # Placeholder; adjust according to actual SOSNet import
        sosnet32 = sosnet32.to(device).eval()

        descriptor = LAFDescriptor(sosnet32, patch_size=32, grayscale_descriptor=True).to(device)

        super().__init__(detector, descriptor)



class KeyNetFeatureSosNet(LocalFeature):
    """Convenience module that combines KeyNet detector + SOSNet descriptor."""

    def __init__(
        self,
        num_features: int = 200,
        upright: bool = False,
        device: torch.device = torch.device("cpu"),
        config: dict = None,
    ) -> None:
        if config is None:
            config = config or {
                    'num_filters': 8,
                    'num_levels': 3,
                    'kernel_size': 5,
                    'Detector_conf': {'nms_size': 5, 'pyramid_levels': 2, 'up_levels': 2, 'scale_factor_levels': 1.3, 's_mult': 12.0},
            }
        # Initialize KeyNet detector
        detector = KeyNetDetector(
            pretrained=True,
            num_features=num_features,
            ori_module=PassLAF() if upright else LAFOrienter(32),
            aff_module=LAFAffNetShapeEstimator(preserve_orientation=upright).eval(),
            keynet_conf=config,
        ).to(device).eval()

        # Initialize SOSNet descriptor
        sosnet32 = SOSNet(pretrained=True).to(device).eval()        
        descriptor = LAFDescriptor(sosnet32, patch_size=32, grayscale_descriptor=True).to(device).eval()

        # Call the parent constructor
        super().__init__(detector, descriptor)

class KeyNetFeatureSIFT(LocalFeature):
    """Convenience module that combines KeyNet detector + SIFT descriptor."""

    def __init__(
        self,
        num_features: int = 200,
        upright: bool = False,
        device: torch.device = torch.device("cpu"),
        config: dict = None,
    ) -> None:
        if config is None:
            config = config or {
                    'num_filters': 8,
                    'num_levels': 3,
                    'kernel_size': 5,
                    'Detector_conf': {'nms_size': 5, 'pyramid_levels': 2, 'up_levels': 2, 'scale_factor_levels': 1.3, 's_mult': 12.0},
            }
             
        # Initialize KeyNet detector
        detector = KeyNetDetector(
            pretrained=True,
            num_features=num_features,
            ori_module=PassLAF() if upright else LAFOrienter(32),
            aff_module=LAFAffNetShapeEstimator(preserve_orientation=upright).eval(),
            keynet_conf=config,
        ).to(device).eval()

        patch_size = 13   
        # Initialize SIFT descriptor
        sift_descriptor = kornia.feature.SIFTDescriptor(patch_size=patch_size, rootsift=True).to(device)

        descriptor = LAFDescriptor(sift_descriptor, patch_size=patch_size, grayscale_descriptor=True).to(device)

        # Call the parent constructor
        super().__init__(detector, descriptor)

In [3]:
from typing import Optional, Tuple
import torch
from torch import nn, Tensor
from kornia.feature import SOSNet, MultiResolutionDetector, LAFDescriptor, LAFOrienter, LocalFeature
from external.REKD import REKD
from config import get_config_rekd

class REKDSosNet(LocalFeature):
    """Combina REKD detector e SOSNet descriptor"""
    def __init__(
        self,
        num_features: int = 200,
        upright: bool = False,
        device: torch.device = torch.device("cpu"),
        config: dict = None,
    ) -> None:
        super().__init__(detector=None, descriptor=None)
        self.device = device
        self.config = config or {
            'num_filters': 8,
            'num_levels': 3,
            'kernel_size': 5,
            'Detector_conf': {'nms_size': 5, 'pyramid_levels': 0, 'up_levels': 0, 'scale_factor_levels': 1.3, 's_mult': 12.0},
        }
        args = get_config_rekd(jupyter=True)
        args.load_dir = 'trained_models/release_group36_f2_s2_t2.log/best_model.pt'
        self.detector = self._initialize_detector(num_features,args)
        self.descriptor = self._initialize_descriptor()

    def _initialize_detector(self,num_features,args):
        """Cria o detector REKD"""
        class REKDetector(nn.Module):
            def __init__(self, args, device: torch.device) -> None:
                super().__init__()
                self.model = REKD(args, device).to(device).eval()
                self.model.load_state_dict(torch.load(args.load_dir, weights_only=False))

            def forward(self, x: Tensor) -> Tensor:
                return self.model(x)[0]  # Apenas as chaves detectadas

        return MultiResolutionDetector(REKDetector(args, self.device), num_features=num_features,config=self.config["Detector_conf"], 
                                       ori_module=LAFOrienter(19)).to(self.device)

    def _initialize_descriptor(self) -> LAFDescriptor:
        """Cria o descritor SOSNet"""
        return LAFDescriptor(SOSNet(pretrained=True).to(self.device).eval(), patch_size=32, grayscale_descriptor=True).to(self.device)

    # def forward(self, img: Tensor, mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor, Tensor]:
    #     """Detecta e descreve pontos-chave"""
    #     lafs, responses = self.detector(img)
    #     return lafs, responses, self.descriptor(img, lafs)

In [4]:
from typing import Optional, Tuple
import torch
from torch import nn, Tensor
from kornia.feature import SOSNet, MultiResolutionDetector, LAFDescriptor, LAFOrienter, LocalFeature
from config import get_config_singular
from best.singular_point import SingularPoints

class SingularPointSosNet(LocalFeature):
    def __init__(        self,
        num_features: int = 200,
        upright: bool = False,
        device: torch.device = torch.device("cpu"),
        config: dict = None
        ) -> None:
        super().__init__(detector=None, descriptor=None)
        self.device = device
        self.config = config or {
            'num_filters': 8,
            'num_levels': 3,
            'kernel_size': 5,
            'Detector_conf': {'nms_size': 5, 'pyramid_levels': 0, 'up_levels': 0, 'scale_factor_levels': 1.3, 's_mult': 12.0},
        }

        args = get_config_singular(jupyter=True)
        args.num_channels = 1
        args.load_dir = './data/models/sp_map_fo_30.pth'

        self.detector = self._initialize_detector(num_features,args)
        self.descriptor = self._initialize_descriptor()
        self.to(self.device)

    def _initialize_detector(self,num_features, args):
        """Cria o detector REKD"""
        class SingularPointDetector(nn.Module):
            
            def __init__(self,args) -> None:
                super().__init__()
                self.model = SingularPoints(args)
                self.model.load_state_dict(torch.load(args.load_dir, weights_only=False))

            def forward(self, x):
                features_key,features_key_summary,features_ori,features_ori_summary,max_coords_values, max_map= self.model(x)
                return features_key_summary

        return MultiResolutionDetector(SingularPointDetector(args), num_features = num_features,config = self.config["Detector_conf"], 
                                       ori_module=LAFOrienter(19))

    def _initialize_descriptor(self) -> LAFDescriptor:
        """Cria o descritor SOSNet"""
        return LAFDescriptor(SOSNet(pretrained=True), patch_size=32, grayscale_descriptor=True).to(self.device)

    # def forward(self, img: Tensor, mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor, Tensor]:
    #     """Detecta e descreve pontos-chave"""
    #     lafs, responses = self.detector(img)
    #     return lafs, responses, self.descriptor(img, lafs)

In [5]:
import torch
import numpy as np
import kornia
from kornia.feature import DescriptorMatcher

class LocalComparisonPipeline:
    """
    Compara imagens de inspeção e referência usando características locais e correspondência de descritores.
    """
    def __init__(self, local_feature, descriptor_matcher):
        self.local_feature = local_feature
        self.descriptor_matcher = descriptor_matcher

    def run(self, inspection_images: torch.Tensor, reference_images: torch.Tensor) -> np.ndarray:
        """
        Compara imagens de inspeção e referência e retorna uma matriz de pontuação baseada em correspondências de descritores.
        """
        n, m = inspection_images.shape[0], reference_images.shape[0]
        scores = np.zeros((n, m))
        cache_reference = {}

        for i_index, i_image in enumerate(inspection_images):
            lafs0, _, descriptors0 = self.local_feature(i_image[:1][None])

            for r_index, r_image in enumerate(reference_images):
                if r_index not in cache_reference:
                    _, _, descriptors1 = self.local_feature(r_image[:1][None])
                    cache_reference[r_index] = descriptors1
                descriptors1 = cache_reference[r_index]

                _, matches = self.descriptor_matcher(descriptors0[0], descriptors1[0])

                num_match = matches.shape[0]
                scores[i_index, r_index] = num_match if num_match >= 3 else 0

        return scores

    def evaluate_matches(self, matches_matrix, threshold=0.5):
        """
        Avalia correspondências entre imagens e calcula TP, FP e FN com base no limiar de similaridade.
        """
        n, m = matches_matrix.shape
        TP, FP, FN = 0, 0, 0

        for i in range(m):
            if np.max(matches_matrix[i]) >= threshold and np.argmax(matches_matrix[i]) == i:
                TP += 1
            else:
                FN += 1

        for i in range(m, n):
            if np.max(matches_matrix[i]) >= threshold and np.argmax(matches_matrix[i]) < m:
                FP += 1

        return TP, FP, FN


In [6]:
class AugmentationGenerator:
    def __init__(self, n_variations):
        # Definir as augmentações
        aug_gen = kornia.augmentation.AugmentationSequential(
            kornia.augmentation.RandomAffine(degrees=360, translate=(0.25, 0.25), scale=(0.95, 1.05), shear=10, p=0.85),
            kornia.augmentation.RandomPerspective(0.3, p=0.85),
            kornia.augmentation.RandomBoxBlur((3, 3), p=0.85),
            data_keys=["input"],
            same_on_batch=True,
        )


        self.augmentation_sequence = aug_gen
        self.n_variations = n_variations
        self.param_list = []
        self.current_index = 0

    def generate_variations(self, input_tensor):
        """
        Gera múltiplas variações de augmentações e coleta seus parâmetros.
        """
        for _ in range(self.n_variations):
            self.augmentation_sequence(input_tensor)            
            self.param_list.append(self.augmentation_sequence._params)

    def __iter__(self):
        self.current_index = 0  # Resetar o índice a cada nova iteração
        return self

    def __next__(self):
        """
        Retorna a próxima variação de parâmetros de augmentação.
        A iteração será circular.
        """
        result = self.param_list[self.current_index]
        self.current_index = (self.current_index + 1) % len(self.param_list)
        return result
    
    def reset(self):
        """Método para resetar o estado do gerador de augmentação."""
        self.current_index = 0  # Reseta o índice de iteração


In [ ]:
import torch
import kornia as K
import logging
from itertools import product
from teste_util import read_dataload_woods,read_dataload_fibers,read_dataload_flower
from utils.TensorImgIO import imshow_points
from kornia.feature import LocalFeature
import warnings
warnings.filterwarnings("ignore")

# Logger específico
logger = logging.getLogger("Main")  # Logger para a execução principal
logger.setLevel(logging.INFO)  # Define o nível de log como INFO

# Definição do grid de parâmetros
param_grid = {
    'num_features': [60],  # Número de características
    'feature_local': [KeyNetFeatureSIFT,KeyNetFeatureSosNet,REKDSosNet,SingularPointSosNet],  # Diversos detectores de características
    'distance': [0.90],  # Distância para a correspondência (padrão 0.8)
    'threshold': [5]  # Limite para correspondência
}

# Função para configurar o detector e executar o código para uma combinação de parâmetros
def run_experiment(dataloader, pipeline,threshold,aug_gen):
    TP_total, FP_total, FN_total = 0, 0, 0  # Inicializa os contadores totais de TP, FP e FN

    for inspection_batch, _ in dataloader:
        inspection_batch = inspection_batch.to(device)        
        params_item = next(aug_gen)
        reference_batch_t =aug_gen.augmentation_sequence(inspection_batch,params=params_item)
        # Pega apenas a metade inicial de reference_batch_t
        half_reference_batch = reference_batch_t[:reference_batch_t.shape[0] // 2]
        logger.debug(f"Lote inspecao: {inspection_batch.shape} Lote referencia {half_reference_batch.shape}")

        # Executa o pipeline de detecção de características
        with torch.no_grad():
            scores = pipeline.run(inspection_batch, half_reference_batch)
            TP, FP, FN = pipeline.evaluate_matches(scores,threshold=threshold)

            # Acumula os resultados de TP, FP e FN
            TP_total += TP
            FP_total += FP
            FN_total += FN
            
            logger.debug(f"TP: {TP}, FP: {FP}, FN: {FN}")
    
    # Exibe os resultados totais de TP, FP e FN após o experimento
    logger.debug(f"Total de TP: {TP_total}, FP: {FP_total}, FN: {FN_total}")
    return TP_total, FP_total, FN_total

if __name__ == "__main__":
    set_seed(42)
    # Inicializa o dispositivo (GPU ou CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logger.info(f"Dispositivo: {device}")
    # 
    # args = get_config_singular(jupyter=True)

    # trainloader, testloader = read_dataload_woods(120,batch_size=132)
    trainloader, testloader = read_dataload_flower(120,batch_size=132)
    # trainloader, testloader = read_dataload_fibers(120,batch_size=132)
    # gerar variacao de transformacoes pespectivas e fotometrica
    iterator = iter(testloader)
    img, labels = next(iterator)
    aug_gen = AugmentationGenerator(10)
    aug_gen.generate_variations(img)

    # Itera entre todas as combinações de parâmetros
    for num_features, feature_local_class, distance, threshold in product(*param_grid.values()):
        logger.info(f"Executando experimento com: {num_features} características, "
            f"detector {feature_local_class.__name__}, "
            f"distância {distance}, e limiar {threshold}")
        set_seed(1)
    
        # Executa o pipeline de detecção de características
        with torch.no_grad():
            # Inicializa o detector com a configuração
            local_feature = feature_local_class(num_features=num_features, device=device).to(device).eval()
            descriptor_matcher = DescriptorMatcher(match_mode="snn", th=distance).to(device).eval()  # Usando o matcher SNN
            pipeline = LocalComparisonPipeline(local_feature=local_feature, descriptor_matcher=descriptor_matcher)
        
            # Executa o experimento e retorna os totais de TP, FP e FN
            TP_total, FP_total, FN_total = run_experiment(testloader, pipeline,threshold, aug_gen)
            aug_gen.reset()# //TODO: reiniciar o gerador de transformacoes
            # //FIXME: o TP_total e FN_total devem ser 0
            check_and_clear_memory()
            # Calcula a precisão, recall e F1-score
            precision = TP_total / (TP_total + FP_total) if (TP_total + FP_total) > 0 else 0
            recall = TP_total / (TP_total + FN_total) if (TP_total + FN_total) > 0 else 0
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            logger.info(f"TP: {TP_total}, FP: {FP_total}, FN: {FN_total}")
            logger.info(f"Precisão: {precision:.2f}, Recall: {recall:.2f}, F1-score: {f1_score:.2f}")

INFO:Main:Dispositivo: cuda


6072


INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 5
INFO:Main:TP: 1354, FP: 3036, FN: 1682
INFO:Main:Precisão: 0.31, Recall: 0.45, F1-score: 0.36
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 1549, FP: 1269, FN: 1487
INFO:Main:Precisão: 0.55, Recall: 0.51, F1-score: 0.53
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 5


------------------------------------------------
'Detector_conf': {'nms_size': 5, 'pyramid_levels': 0, 'up_levels': 0, 'scale_factor_levels': 1.3, 's_mult': 12.0},
aug_gen = kornia.augmentation.AugmentationSequential(
    kornia.augmentation.RandomAffine(degrees=360, translate=(0.25, 0.25), scale=(0.95, 1.05), shear=10, p=0.8),
    kornia.augmentation.RandomPerspective(0.3, p=0.7),
    kornia.augmentation.RandomBoxBlur((3, 3), p=0.7),
    data_keys=["input"],
    same_on_batch=True,
)

FLOWERS

INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 0.5
INFO:Main:TP: 712, FP: 0, FN: 308
INFO:Main:Precisão: 1.00, Recall: 0.70, F1-score: 0.82
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 1002, FP: 0, FN: 18
INFO:Main:Precisão: 1.00, Recall: 0.98, F1-score: 0.99
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 1013, FP: 0, FN: 7
INFO:Main:Precisão: 1.00, Recall: 0.99, F1-score: 1.00
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 1020, FP: 0, FN: 0
INFO:Main:Precisão: 1.00, Recall: 1.00, F1-score: 1.00


------------------------------------------------
'Detector_conf': {'nms_size': 5, 'pyramid_levels': 0, 'up_levels': 0, 'scale_factor_levels': 1.3, 's_mult': 12.0},
aug_gen = kornia.augmentation.AugmentationSequential(
    kornia.augmentation.RandomAffine(degrees=360, translate=(0.25, 0.25), scale=(0.95, 1.05), shear=10, p=0.85),
    kornia.augmentation.RandomPerspective(0.3, p=0.85),
    kornia.augmentation.RandomBoxBlur((3, 3), p=0.85),
    data_keys=["input"],
    same_on_batch=True,
)
FLOWERS

INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 0.5
INFO:Main:TP: 690, FP: 0, FN: 330
INFO:Main:Precisão: 1.00, Recall: 0.68, F1-score: 0.81
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 983, FP: 0, FN: 37
INFO:Main:Precisão: 1.00, Recall: 0.96, F1-score: 0.98
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 1011, FP: 0, FN: 9
INFO:Main:Precisão: 1.00, Recall: 0.99, F1-score: 1.00
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 1020, FP: 0, FN: 0
INFO:Main:Precisão: 1.00, Recall: 1.00, F1-score: 1.00

------------------------------------------------
'Detector_conf': {'nms_size': 5, 'pyramid_levels': 2, 'up_levels': 2, 'scale_factor_levels': 1.3, 's_mult': 12.0},#KeyNetFeatureSIFT,KeyNetFeatureSosNet
'Detector_conf': {'nms_size': 5, 'pyramid_levels': 0, 'up_levels': 0, 'scale_factor_levels': 1.3, 's_mult': 12.0},#REKD e SingularPointsDetector
aug_gen = kornia.augmentation.AugmentationSequential(
    kornia.augmentation.RandomAffine(degrees=360, translate=(0.25, 0.25), scale=(0.95, 1.05), shear=10, p=0.85),
    kornia.augmentation.RandomPerspective(0.3, p=0.85),
    kornia.augmentation.RandomBoxBlur((4, 4), p=0.85),
    data_keys=["input"],
    same_on_batch=True,
)

WOODS
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 0.5
INFO:Main:TP: 31, FP: 0, FN: 465
INFO:Main:Precisão: 1.00, Recall: 0.06, F1-score: 0.12
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 115, FP: 0, FN: 381
INFO:Main:Precisão: 1.00, Recall: 0.23, F1-score: 0.38
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 246, FP: 0, FN: 250
INFO:Main:Precisão: 1.00, Recall: 0.50, F1-score: 0.66
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 387, FP: 0, FN: 109
INFO:Main:Precisão: 1.00, Recall: 0.78, F1-score: 0.88

FIBERS

INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 0.5
INFO:Main:TP: 34, FP: 0, FN: 186
INFO:Main:Precisão: 1.00, Recall: 0.15, F1-score: 0.27
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 8, FP: 0, FN: 212
INFO:Main:Precisão: 1.00, Recall: 0.04, F1-score: 0.07
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 4, FP: 0, FN: 216
INFO:Main:Precisão: 1.00, Recall: 0.02, F1-score: 0.04
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 111, FP: 0, FN: 109
INFO:Main:Precisão: 1.00, Recall: 0.50, F1-score: 0.67

FLOWERS

INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 0.5
INFO:Main:TP: 390, FP: 0, FN: 630
INFO:Main:Precisão: 1.00, Recall: 0.38, F1-score: 0.55
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 513, FP: 0, FN: 507
INFO:Main:Precisão: 1.00, Recall: 0.50, F1-score: 0.67
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 990, FP: 0, FN: 30
INFO:Main:Precisão: 1.00, Recall: 0.97, F1-score: 0.99
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 0.5
INFO:Main:TP: 1009, FP: 0, FN: 11
INFO:Main:Precisão: 1.00, Recall: 0.99, F1-score: 0.99

------------------------------------------------
aug_gen = kornia.augmentation.AugmentationSequential(
    kornia.augmentation.RandomAffine(degrees=360, translate=(0.25, 0.25), scale=(0.95, 1.05), shear=10, p=0.85),
    kornia.augmentation.RandomPerspective(0.3, p=0.85),
    kornia.augmentation.RandomBoxBlur((3, 3), p=0.85),
    data_keys=["input"],
    same_on_batch=True,
)


FLOWERS

INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 5
INFO:Main:TP: 259, FP: 510, FN: 251
INFO:Main:Precisão: 0.34, Recall: 0.51, F1-score: 0.41
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 293, FP: 122, FN: 217
INFO:Main:Precisão: 0.71, Recall: 0.57, F1-score: 0.63
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 506, FP: 171, FN: 4
INFO:Main:Precisão: 0.75, Recall: 0.99, F1-score: 0.85
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 510, FP: 165, FN: 0
INFO:Main:Precisão: 0.76, Recall: 1.00, F1-score: 0.86


WOODS

INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 5
INFO:Main:TP: 14, FP: 330, FN: 311
INFO:Main:Precisão: 0.04, Recall: 0.04, F1-score: 0.04
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 124, FP: 151, FN: 201
INFO:Main:Precisão: 0.45, Recall: 0.38, F1-score: 0.41
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 176, FP: 191, FN: 149
INFO:Main:Precisão: 0.48, Recall: 0.54, F1-score: 0.51
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 292, FP: 50, FN: 33
INFO:Main:Precisão: 0.85, Recall: 0.90, F1-score: 0.88

FIBERS

INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 5
INFO:Main:TP: 25, FP: 198, FN: 173
INFO:Main:Precisão: 0.11, Recall: 0.13, F1-score: 0.12
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 8, FP: 198, FN: 190
INFO:Main:Precisão: 0.04, Recall: 0.04, F1-score: 0.04
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 2, FP: 21, FN: 196
INFO:Main:Precisão: 0.09, Recall: 0.01, F1-score: 0.02
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 122, FP: 56, FN: 76
INFO:Main:Precisão: 0.69, Recall: 0.62, F1-score: 0.65

In [ ]:
import numpy as np

# Definindo o valor do limiar
threshold = 1.0

# Matriz de similaridade (5x5)
matches_matrix = np.array([
    [5.0, 0.2, 0.3, 0.0, 0.0],  # Linha 0 (correspondência correta com o próprio item)
    [0.1, 3.0, 0.4, 0.0, 0.0],  # Linha 1 (correspondência correta com o próprio item)
    [0.2, 0.3, 3.0, 0.0, 0.0],  # Linha 2 (correspondência correta com o próprio item)
    [0.0, 0.0, 0.0, 0.0, 0.0],  # Linha 3 (nenhuma correspondência correta)
    [0.0, 0.0, 0.0, 0.0, 0.0],   # Linha 4 (nenhuma correspondência correta)
    [0.0, 3.0, 0.0, 0.0, 0.0],  # Linha 5 (nenhuma correspondência correta)
    [0.0, 0.0, 1.0, 0.0, 0.0]   # Linha 7 (nenhuma correspondência correta)
])


# Testando a função
class Evaluator:
    def evaluate_matches(self, matches_matrix, threshold=0.5):
        n, m = matches_matrix.shape
        TP, FP, FN = 0, 0, 0

        for i in range(m):
            if np.max(matches_matrix[i]) >= threshold and np.argmax(matches_matrix[i]) == i:
                TP += 1
            else:
                FN += 1

        for i in range(m, n):
            if np.max(matches_matrix[i]) >= threshold and np.argmax(matches_matrix[i]) < m:
                FP += 1

        return TP, FP, FN

# Instanciando o avaliador e chamando a função
evaluator = Evaluator()
TP, FP, FN = evaluator.evaluate_matches(matches_matrix, threshold)

# Resultado
print("True Positives (TP):", TP)
print("False Positives (FP):", FP)
print("False Negatives (FN):", FN)
